# Google Earth Engine

In [ ]:
# google earth engine should work if installed, but it needs authentication
# making datasets from it with a computer with internet may be preferable to using it while on a compute node
# import ee

# # Trigger the authentication flow.
# ee.Authenticate()

# # Initialize the library.
# ee.Initialize(project='my-project')

# GBIF

In [3]:
from pygbif import species as species
from pygbif import occurrences as occ

In [4]:
splist = ['Cyanocitta stelleri', 'Junco hyemalis', 'Aix sponsa',
  'Ursus americanus', 'Pinus conorta', 'Poa annuus']
keys = [ species.name_backbone(x)['usageKey'] for x in splist ]

In [5]:
out = [ occ.search(taxonKey = x, limit=0)['count'] for x in keys ]

In [6]:
x = dict(zip(splist, out))
sorted(x.items(), key=lambda z:z[1], reverse=True)

[('Junco hyemalis', 14827936),
 ('Aix sponsa', 6498428),
 ('Cyanocitta stelleri', 2756192),
 ('Poa annuus', 741730),
 ('Pinus conorta', 105934),
 ('Ursus americanus', 53946)]

# SatCLIP

In [7]:
import sys
import os

In [11]:
# clone satclip into a folder, use your own username
satclip_path = "/home/mila/g/<username>/scratch/satclip"

In [12]:
sys.path.append(satclip_path)
sys.path.insert(0, "/home/mila/g/<username>/scratch/satclip/satclip")

In [13]:
from satclip.model import SatCLIP

/network/scratch/g/gregory.bell/COMP-685-Fall-2025/.venv/lib/python3.11/site-packages/torch/__init__.py:1617: UserWarning: Please use the new API settings to control TF32 behavior, such as torch.backends.cudnn.conv.fp32_precision = 'tf32' or torch.backends.cuda.matmul.fp32_precision = 'ieee'. Old settings, e.g, torch.backends.cuda.matmul.allow_tf32 = True, torch.backends.cudnn.allow_tf32 = True, allowTF32CuDNN() and allowTF32CuBLAS() will be deprecated after Pytorch 2.9. Please see https://pytorch.org/docs/main/notes/cuda.html#tensorfloat-32-tf32-on-ampere-and-later-devices (Triggered internally at /pytorch/aten/src/ATen/Context.cpp:80.)
  _C._set_float32_matmul_precision(precision)


In [14]:
from satclip.location_encoder import LocationEncoder

In [15]:
model = SatCLIP(
    embed_dim=512,
    image_resolution=224, in_channels=13, vision_layers=4, vision_width=768, vision_patch_size=32, # Image encoder
    le_type='sphericalharmonics', pe_type='siren', legendre_polys=10, frequency_num=16, max_radius=360, min_radius=1, harmonics_calculation='analytic'  # Location encoder
)


using vision transformer


In [16]:
import torch

In [17]:
img_batch = torch.randn(32, 13, 224, 224) # Represents a batch of 32 images
loc_batch = torch.randn(32, 2) # Represents the corresponding 32 locations (lon/lat)

In [18]:
with torch.no_grad():
    logits_per_image, logits_per_coord = model(img_batch, loc_batch)
    probs = logits_per_image.softmax(dim=-1).detach().cpu().numpy()

# INaturalist

In [19]:
from pyinaturalist import get_observations

In [20]:
observations = get_observations()

INFO:Request:
GET https://api.inaturalist.org/v1/observations
User-Agent: python-requests/2.32.5 pyinaturalist/0.20.2
Accept-Encoding: gzip, deflate
Accept: application/json
Connection: keep-alive



In [21]:
observations.keys()

dict_keys(['total_results', 'page', 'per_page', 'results'])

In [22]:
observations['total_results']

312784440

# eBird

In [1]:
# account api key is needed, may be best to load dataset outside computer node.

# TimesFM

In [1]:
# clone in another folder and install in the uv venv

In [1]:
# pip install transformers>=4.53 torch numpy
import numpy as np
import torch
from torch.nn.functional import normalize
from transformers import TimesFmModelForPrediction

# 1) Load a TimesFM checkpoint (univariate)
model = TimesFmModelForPrediction.from_pretrained(
    "google/timesfm-2.0-500m-pytorch",  # or a newer 2.5 checkpoint if/when available
    dtype=torch.bfloat16,
    attn_implementation="sdpa",
    device_map="auto",
)

# 2) Example batch of variable-length time series (replace with your data)
series = [
    np.sin(np.linspace(0, 20, 100)),   # length 100
    np.sin(np.linspace(0, 20, 240)),   # length 240
    np.sin(np.linspace(0, 20, 512)),   # length 512
    np.sin(np.linspace(0, 20, 1024)),   # length 1024
]

# Convert to tensors on the model's device
past_values = [torch.tensor(x, dtype=torch.bfloat16, device=model.device) for x in series]

# TimesFM uses a small frequency embedding; if you don't know it, you can set zeros.
# Shape: (batch,)
freq = torch.zeros(len(past_values), dtype=torch.long, device=model.device)

# 3) Forward pass requesting hidden states
with torch.no_grad():
    out = model(
        past_values=past_values,
        freq=freq,
        output_hidden_states=True,   # <— important
        return_dict=True,
    )

# out.last_hidden_state: (batch, seq_len, hidden_size) after internal padding
# We'll mean-pool over the *valid* timesteps for each series, then L2-normalize.

embeddings = []
for i, ts in enumerate(past_values):
    L = ts.shape[0]  # original (unpadded) length
    # slice to valid region, mean-pool over time
    vec = out.last_hidden_state[i, :L, :].float().mean(dim=0)
    embeddings.append(vec)

embeddings = torch.stack(embeddings, dim=0)           # (batch, hidden_size)
embeddings = normalize(embeddings, p=2, dim=1)        # cosine-friendly

print(embeddings.shape)   # e.g., torch.Size([4, 1280]) for the 500M checkpoint

/network/scratch/g/gregory.bell/COMP-685-Fall-2025/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


torch.Size([4, 1280])


# Chronos

In [2]:

import numpy as np
import torch
from torch.nn.functional import normalize
from chronos import ChronosPipeline

# 1) Load a Chronos (T5) checkpoint
pipe = ChronosPipeline.from_pretrained(
    "amazon/chronos-t5-small",           # try base/large for bigger models
    device_map="auto",
    torch_dtype=torch.bfloat16 if torch.cuda.is_available() else torch.float32,
)

# 2) Example batch: variable-length univariate series
rng = np.random.default_rng(0)
series = [
    np.sin(np.linspace(0, 8*np.pi, 120)) + 0.1 * rng.standard_normal(120),
    np.sin(np.linspace(0, 8*np.pi, 200)) + 0.1 * rng.standard_normal(200),
    np.sin(np.linspace(0, 8*np.pi, 350)) + 0.1 * rng.standard_normal(350),
]

# Convert to a list of 1D tensors (no manual padding needed)
batch = [torch.tensor(x, dtype=torch.float32) for x in series]
lengths = [len(x) for x in batch]

# 3) Extract encoder embeddings for the whole batch
#    emb has shape (batch, seq_len, hidden) with internal padding handled for you.
emb, tok_state = pipe.embed(batch)

# 4) Pool to a single fixed-size vector per series (mask out padding)
#    We'll mean-pool over the valid (unpadded) timesteps, then L2-normalize.
pooled = []
for i, L in enumerate(lengths):
    vec = emb[i, :L, :].float().mean(dim=0)   # (hidden,)
    pooled.append(vec)

embeddings = torch.stack(pooled, dim=0)       # (batch, hidden)
embeddings = normalize(embeddings, p=2, dim=1)

print("Per-series embedding shape:", embeddings.shape)   # e.g., torch.Size([3, hidden])


`torch_dtype` is deprecated! Use `dtype` instead!


Per-series embedding shape: torch.Size([3, 512])
